# CSC 575 Intelligent Information Retrieval
### Siravich Khongrod (In-class)
## CSC 575 HW#4

http://condor.depaul.edu/ntomuro/courses/575/assign/HW4.html  
https://nbviewer.jupyter.org/url/condor.depaul.edu/ntomuro/courses/575/assign/575hw4.ipynb

<p><strong>Overview</strong>:</p>
<p>Implement the 'Inverted Index Retrieval Algorithm' (in
<a href="http://condor.depaul.edu/ntomuro/courses/575/notes/VS-Retrieval.pptx">
lecture note (#6)</a>) and the evaluation metric Mean Average Precision (MAP) 
(in <a href="http://condor.depaul.edu/ntomuro/courses/575/notes/Evaluation.pptx">
lecture note (#7)</a>), and apply to a corpus called <a href="http://ir.dcs.gla.ac.uk/resources/test_collections/">Medline 
collection</a>.&nbsp; 
</p>
<p>The Medline 
collection is one of the Information Retrieval (IR) standard test 
collections, which have been used by many researchers as benchmark to evaluate IR 
systems.&nbsp; It contains 1033 documents (abstracts of papers published on 
Medline), 30 queries and relevance judgments of all query-document pairs.&nbsp; 
</p>

### Programming: Vector-space Retrieval & Evaluation -- Partially filled code

### (1) Step 1: Load Inverted Index (H) and compute DocLen (DL).

In [3]:
import csv
import math

tindexfile = 'medline_term_index.csv'
invindexfile = 'medline_inverted_index.csv'
dindexfile = 'medline_doc_index.csv'

# Number of documents in the corpus (hard-coded for this corpus)
N = 1033

# Major data structures
H_invindex = {} # inverted index; term -> (idf, L:hashmap of (docID . tf))
DL_doclen = {}  # document lengths; docID -> len

## (1) Read the term index file and populate the invindex first
tid2term_map = {} # temporary storage to hold mappings of termID -> term

fin = open(tindexfile, 'r', encoding='utf-8')
reader = csv.reader(fin, delimiter='\t')
for line in reader:
    term = line[0]    # term string
    termID = line[1]  # termID
    df = int(line[2]) # document frequency
    idf = math.log10(N/df) # idf
    # record term -> (idf, emptyL) in H
    H_invindex[term] = (idf, dict())
    # record termID -> term 
    tid2term_map[termID] = term 
fin.close()

## (2) Read the inverted index file and add postings lists in H.
## Also compute document lengths too, incrementally -- and record in DL.
fin = open(invindexfile, 'r')
reader = csv.reader(fin, delimiter='\t')
for line in reader:
    termID = line[0]
    idx = 1
    while idx < (len(line)-1):
        docID = line[idx]
        tf = int(line[idx+1]) # raw tf of the term in this document
        # Record docID -> tf in term's L
        L = (H_invindex[tid2term_map[termID]])[1]
        L[docID] = tf  # docID -> raw term frequency
        
        # Accumulate the component vector length for the document
        tfidf = tf * (H_invindex[tid2term_map[termID]])[0] # tf * idf
        tfidfsq = math.pow(tfidf, 2.0)
        if docID in DL_doclen:
            DL_doclen[docID] += tfidfsq
        else:
            DL_doclen[docID] = tfidfsq
        #
        idx += 2
fin.close()

# Fix the DL entries by applying sqrt to make vector length.
for docID in DL_doclen.keys():
    val = DL_doclen[docID]
    DL_doclen[docID] = math.sqrt(val)

    
print ('Total # terms: %d' % len(H_invindex))
for term in ['pentobarbit', 'defici', 'treatment']:
    print (' - Entry for \'%s\': df=%s, idf=%s' % (term, len(H_invindex[term][1]), H_invindex[term][0]))

print ('\nTotal # documents: %d' % len(DL_doclen))
for docID in ['59', '1033']:
    print (' - Vector len for Doc %s = %s' % (docID, DL_doclen[docID]))


Total # terms: 11463
 - Entry for 'pentobarbit': df=4, idf=2.412040330191658
 - Entry for 'defici': df=39, idf=1.4230357144931214
 - Entry for 'treatment': df=172, idf=0.7785718746120717

Total # documents: 1033
 - Vector len for Doc 59 = 13.811725366348801
 - Vector len for Doc 1033 = 31.163653356034512


### (2) Step 2: Queries as Vectors

In [4]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

queryfile = 'medline.query'

# A list of queries. Each Query is a tuple, (qID, Q:term->tf map)
Queries_list = []

fin = open(queryfile, 'r', encoding='utf-8')#'iso-8859-1')
porter = nltk.PorterStemmer()

for line in fin:
    matchObj = re.match(r'^(\d+)\s+(.*)', line)
    if not matchObj:
        print ("ERROR with line -- %s" % line)
    else:
        queryID = matchObj.group(1) # queryID
        text = matchObj.group(2)    # query string (ignoring sentences)

        # process text string -- same processing as one applied to documents.
        tokens = word_tokenize(text.lower())
        terms = [porter.stem(w) for w in tokens if w not in stopwords.words('english') and len(w) > 1] # (**)
        # term frequencies of the terms in this query are obtained by NLTK's FreqDist
        fdist = nltk.FreqDist(terms)
        # append the Query in the list
        Queries_list.append((queryID, dict(fdist)))
fin.close()

print ('Total # queries: %d' % len(Queries_list))
for qid in [1, 21]:
    print (' - Query %s: %s' % (Queries_list[qid][0], Queries_list[qid][1]))

Total # queries: 30
 - Query 2: {'relationship': 1, 'blood': 1, 'cerebrospin': 1, 'fluid': 1, 'oxygen': 1, 'concentr': 1, 'partial': 1, 'pressur': 1, 'method': 1, 'interest': 1, 'polarographi': 1}
 - Query 22: {'mycoplasma': 1, 'infect': 1, 'presenc': 1, 'embryo': 1, 'fetu': 1, 'newborn': 1, 'infant': 1, 'anim': 1, 'pregnanc': 1, 'gynecolog': 1, 'diseas': 1, 'relat': 1, 'chromosom': 2, 'abnorm': 1}


In [5]:
print ('Total # terms: %d' % len(H_invindex))
for term in ['pentobarbit']:
    print (' - Entry for \'%s\': df=%s, idf=%s' % (term, len(H_invindex[term][1]), H_invindex[term][0]))
    print(H_invindex[term])
    print()

print ('\nTotal # documents: %d' % len(DL_doclen))
for docID in ['59', '1033']:
    print (' - Vector len for Doc %s = %s' % (docID, DL_doclen[docID]))

Total # terms: 11463
 - Entry for 'pentobarbit': df=4, idf=2.412040330191658
(2.412040330191658, {'187': 1, '301': 1, '416': 1, '419': 1})


Total # documents: 1033
 - Vector len for Doc 59 = 13.811725366348801
 - Vector len for Doc 1033 = 31.163653356034512


## (3) Retrieval and Ranking -- Step 3,4,5 for each Query
 - For each query, follow Step 3,4,5 of the Vector Space Retrieval Algorithm and obtain a ranked list of retrieved documents (sorted by the cosine measure) -- 'R' in the algorithm.  
 - Then save the ranked lists in a list (in the same order of the query) -- to be used in the next step/Evaluation.
 - (**) Also, WRITE the ranked lists to an output file.  See the homework page for details.  
 
### Write the ranked list of documents (sorted by the descending order of cosine) to an output file.
* Name the file "rankedlist.txt".
* For each query, write the query ID number, followed by the document ID numbers in the rank order, in one line, separated by a comma.  For example the beginning part of the first three lines should look like:  
1, 72, 500, 965, 360, 171, 15, 166, 181, 513, 511, ..  
2, 258, 712, 289, 162, 299, 187, 237, 96, 291, 192, ..  
3, 70, 62, 160, 286, 71, 230, 234, 407, 276, 59, ..  

In [6]:
i = 0
for key in H_invindex:
    print(key+' '+str(H_invindex[key]))
    if(i==5):break
    i+=1



'' (1.4230357144931214, {'497': 1, '545': 1, '556': 1, '560': 1, '565': 1, '592': 2, '602': 2, '604': 4, '606': 1, '608': 6, '611': 1, '612': 2, '613': 1, '615': 2, '624': 1, '625': 1, '630': 2, '633': 2, '640': 1, '647': 1, '660': 1, '678': 1, '691': 1, '699': 1, '709': 1, '713': 1, '715': 2, '720': 2, '738': 1, '740': 3, '743': 1, '750': 1, '752': 1, '789': 1, '798': 1, '801': 1, '807': 1, '808': 3, '817': 2})
'a (2.7130703258556395, {'421': 1, '609': 1})
'achondroplast (3.0141003215196207, {'576': 1})
'adequ (3.0141003215196207, {'421': 1})
'agnos (3.0141003215196207, {'358': 2})
'air (3.0141003215196207, {'70': 1})


In [223]:
# RETRIEVE IDF IN QUERY
from collections import defaultdict
from scipy.spatial import distance
import numpy as np
import pandas as pd
docs_tfidf = defaultdict(list)
i=0
f=open('rankedlist.txt','w')

############### For each token, T, in Q: ###############
for q in Queries_list:
#     doc_w_terms=defaultdict(list)
#     doc_w_terms=[]
    q_tfidf=[]
    doc_score={}
########################################################################################################################
### Let Ibe the IDF of T, and Kbe the frequency of T in Q;
### Set the weight of T in Q: W= K*I; (tf*idffor T in Q)
### If a term in a query was not in the vocabulary (the dictionary structure made from the documents), ignore/skip it.
    for token in (q[1].keys()): # ITERATE OVER QUERY TERMS
        if(token not in H_invindex.keys()):
            continue
#         print("QUERY TERM: "+token+' '+str(H_invindex[token][0])+'x'+str(q[1][token]))
        qt_tfidf=H_invindex[token][0]*q[1][token]
        q_tfidf.append(qt_tfidf)
#         print("DOCS: ",end="")
#         print(H_invindex[token][1])
########################################################################################################################
###     Let L be the posting list of T from H;
###     For each HashMap, O, in L:
        for docid in H_invindex[token][1].keys():
#             print('\t'+str(docid)+': '+str(H_invindex[token][1][docid])+' x '+str(H_invindex[token][0])+' = '
#                   +str(H_invindex[token][1][docid]*H_invindex[token][0]))
########################################################################################################################
###         Increment D’s score by W* C * I; (tf*idffor T in Q x tf*idffor T in D)
            if (docid not in doc_score.keys()):
                doc_score[docid]=0
            doc_score[docid]+=qt_tfidf*H_invindex[token][1][docid]*H_invindex[token][0]
########################################################################################################################
### Compute the length, L, of the vector Q (square-root of the sum of the squares of its weights).
    q_len=sum(np.array(q_tfidf)**2)
### For each retrieved document D in R:
###     Let S be the current accumulated score of D;
###     (S is the dot-product of D and Q)
###     Let Y be the length of D in DL;
###     Let D’s final score to be S/(L * Y); (the cosine)
    for docid in doc_score.keys():
        doc_score[docid] = doc_score[docid] / (q_len*DL_doclen[docID])
    if(i<3):print(q[0], end=" ")
    f.write(q[0]+' ')
    for w in sorted(doc_score, key=doc_score.get, reverse=True):
        if(i<3):print(w,end=' ')
#             print(str(w) + '('+str(doc_score[w])+')', end=" ")
        f.write(w+' ')
    f.write('\n')
    if(i<3):print('\n')

    i+=1
#     if(i==3):break
f.close()

1 500 965 72 15 212 166 513 499 511 360 181 171 182 637 168 509 206 142 172 512 401 13 14 79 164 165 167 169 170 183 184 727 138 180 175 336 549 58 65 570 41 403 542 185 186 211 504 506 510 99 177 640 758 838 259 645 11 173 174 209 256 540 541 642 700 763 876 899 986 112 913 125 127 231 469 213 501 502 503 505 507 508 9 78 130 158 163 178 215 333 404 445 567 578 584 589 590 593 603 652 654 658 660 730 734 816 869 870 873 880 900 905 4 24 35 75 86 100 114 156 160 162 195 196 208 214 220 229 234 253 262 267 300 322 326 342 372 374 383 421 442 454 464 480 485 496 619 627 650 655 685 715 719 726 732 743 744 769 797 799 811 826 831 835 849 863 886 888 896 907 950 981 999 1001 1029 42 67 70 82 83 87 95 108 176 188 189 207 248 282 299 367 375 408 466 486 516 521 557 569 573 576 577 580 581 585 604 606 608 631 632 643 648 664 800 809 812 872 875 909 910 927 984 1020 

2 712 289 258 162 291 187 299 313 243 292 713 418 192 296 293 80 237 297 708 760 715 157 302 420 3 862 301 118 236 974 880 96 6

## (4) Evaluation -- Compute MAP
### Print the MAP score (to the terminal).
 - Read the relevancy answers from the file "medline.rel".
 - Compare the ranked lists with the anwers, and compute the MAP score.
 - (**) Also print the MAP score (to the terminal).

In [167]:
import requests
r=requests.get("http://condor.depaul.edu/ntomuro/courses/575/assign/medline.rel")
r.content
rel_docs={}
for line in r.text.split('\n'):
#     print(line)
    line=(line.strip().split(' '))
    if(line[0]!=''):
        rel_docs[line[0]]=line[1:]
#     print(rel_docs[line[0]])
print(rel_docs)

{'1': ['13', '14', '15', '72', '79', '138', '142', '164', '165', '166', '167', '168', '169', '170', '171', '172', '180', '181', '182', '183', '184', '185', '186', '211', '212', '499', '500', '501', '502', '503', '504', '506', '507', '508', '510', '511', '513'], '2': ['80', '90', '162', '187', '236', '237', '258', '289', '290', '292', '293', '294', '296', '300', '301', '303'], '3': ['59', '62', '67', '69', '70', '71', '73', '78', '81', '160', '163', '230', '231', '232', '233', '234', '276', '277', '279', '282', '283', '287'], '4': ['93', '94', '96', '141', '173', '174', '175', '176', '177', '178', '207', '208', '209', '210', '259', '396', '397', '399', '400', '404', '405', '406', '408'], '5': ['1', '2', '4', '5', '6', '7', '8', '9', '10', '11', '12', '158', '159', '188', '304', '305', '306', '307', '325', '326', '327', '329', '330', '331', '332', '333'], '6': ['112', '115', '116', '118', '122', '238', '239', '242', '260', '309', '320', '321', '323'], '7': ['92', '121', '189', '247', '26

• Consider rank position of each relevantdoc  
    – K1, K2, … KR  
• Compute Precision@Kfor each K1, K2, … KR  
• Average precision = average of P@K  
• Ex: has AvgPrecof  
• MAP is the average of the average precision value for a set of queries.

In [225]:
# Read the file from snippet above
retrived_docs={}
f=open('rankedlist.txt','r')
lines=f.readlines()
for line in lines:
#     print(line)
    line=line.strip().split(' ')
    retrived_docs[line[0]]=line[1:]

In [226]:
print(rel_docs['1'],end='\n\n')
print(retrived_docs['1'])

['13', '14', '15', '72', '79', '138', '142', '164', '165', '166', '167', '168', '169', '170', '171', '172', '180', '181', '182', '183', '184', '185', '186', '211', '212', '499', '500', '501', '502', '503', '504', '506', '507', '508', '510', '511', '513']

['500', '965', '72', '15', '212', '166', '513', '499', '511', '360', '181', '171', '182', '637', '168', '509', '206', '142', '172', '512', '401', '13', '14', '79', '164', '165', '167', '169', '170', '183', '184', '727', '138', '180', '175', '336', '549', '58', '65', '570', '41', '403', '542', '185', '186', '211', '504', '506', '510', '99', '177', '640', '758', '838', '259', '645', '11', '173', '174', '209', '256', '540', '541', '642', '700', '763', '876', '899', '986', '112', '913', '125', '127', '231', '469', '213', '501', '502', '503', '505', '507', '508', '9', '78', '130', '158', '163', '178', '215', '333', '404', '445', '567', '578', '584', '589', '590', '593', '603', '652', '654', '658', '660', '730', '734', '816', '869', '870', 

In [219]:
import csv

i=0
allMap=[]
for qid in rel_docs.keys():
    print('QUERY: '+qid+' : '+str(len(rel_docs[qid]))+' rel docs')
#     print(rel_docs[qid])
    print('RETRIEVED: '+str(len(retrived_docs[qid]))+' docs')
    MAP=[]
    for i in range(0,len(retrived_docs[qid])):
        retrived=set(retrived_docs[qid][:i+1])
#         print('intersection',end=" ")
#         print(retrived.intersection(rel_docs[qid]))
#         print('last element ',end=" ")
#         print(retrived_docs[qid][i])
#         print(retrived_docs[qid][i] in rel_docs[qid])
        if(retrived_docs[qid][i] in rel_docs[qid]):
#             print(len(retrived.intersection(rel_docs[qid]))/len(retrived))
            MAP.append(len(retrived.intersection(rel_docs[qid]))/len(retrived))
#         print(retrived_docs[qid][:i],end='\n')
#     for doc in retrived_docs[qid]:
#         print(doc,end=" ")
    print('MAP: ',end='')
    print(np.mean(MAP))
    allMap.append(np.mean(MAP))
    print()
print('Final MAP')
print(np.mean(allMap))

QUERY: 1 : 37 rel docs
RETRIEVED: 223 docs
MAP: 0.7103648073194891

QUERY: 2 : 16 rel docs
RETRIEVED: 434 docs
MAP: 0.44739041748932595

QUERY: 3 : 22 rel docs
RETRIEVED: 97 docs
MAP: 0.5666123941442229

QUERY: 4 : 23 rel docs
RETRIEVED: 244 docs
MAP: 0.26052017108858716

QUERY: 5 : 26 rel docs
RETRIEVED: 391 docs
MAP: 0.7242445103638524

QUERY: 6 : 13 rel docs
RETRIEVED: 302 docs
MAP: 0.5602793855277783

QUERY: 7 : 15 rel docs
RETRIEVED: 672 docs
MAP: 0.6474262940695009

QUERY: 8 : 11 rel docs
RETRIEVED: 639 docs
MAP: 0.346564986986273

QUERY: 9 : 28 rel docs
RETRIEVED: 434 docs
MAP: 0.39111972597336747

QUERY: 10 : 24 rel docs
RETRIEVED: 39 docs
MAP: 0.5967537883479913

QUERY: 11 : 18 rel docs
RETRIEVED: 316 docs
MAP: 0.590986984505543

QUERY: 12 : 9 rel docs
RETRIEVED: 436 docs
MAP: 0.4596016855017869

QUERY: 13 : 21 rel docs
RETRIEVED: 113 docs
MAP: 0.871748599242188

QUERY: 14 : 16 rel docs
RETRIEVED: 570 docs
MAP: 0.6063345565807665

QUERY: 15 : 29 rel docs
RETRIEVED: 433 docs
MA

### Include your overall comments, for instance, how difficult you felt this assignment was, and any particular difficulties you encountered.  Write in well-written English prose.

Understanding the data structure is really the fundamental of completing this assignment. Not only a simple hashmap or python dictionary is crucial but also taking time to consider various data structures that would result a good design of such retrieval system. 

As for the conceptual ideas behind the algorithm, having a lot of figures to compute might seem like a great challenge, but slowly following each steps from the lecture notes would ease coding and allow the program to be built with good directions
